In [1]:
from random import sample
import random
from tqdm import tqdm
import pandas as pd
import requests
import time
import DHB as dhb
tqdm.pandas()

In [2]:
dhb.db_open()
df = dhb.sql("select * from match_aram")
dhb.db_close()

OPEN
close


In [3]:
df["WIN"] = df.apply(lambda x: 1 if x["WIN"] == "True" else 0 , axis=1)

In [4]:
champ_lst = list(df.groupby("CHAMPIONNAME").count().reset_index()["CHAMPIONNAME"])

# 1. 아이템의 키값과 url 매칭

In [5]:
url = "http://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/item.json"
res = requests.get(url).json()
item_key = list(res['data'].keys())

In [14]:
list(map(lambda x: {x:"http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/"+x+".png"}, item_key))

[{'1001': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1001.png'},
 {'1004': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1004.png'},
 {'1006': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1006.png'},
 {'1011': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1011.png'},
 {'1018': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1018.png'},
 {'1026': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1026.png'},
 {'1027': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1027.png'},
 {'1028': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1028.png'},
 {'1029': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1029.png'},
 {'1031': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1031.png'},
 {'1033': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1033.png'},
 {'1035': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1035.png'},
 {'1036': 'http://ddragon.leagueoflegends.com/cdn/11

In [6]:
itemurl = list(map(lambda x: "http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/"+x+".png", item_key))

In [7]:
item_lst = dict(zip(item_key,itemurl))

In [8]:
item_lst

{'1001': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1001.png',
 '1004': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1004.png',
 '1006': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1006.png',
 '1011': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1011.png',
 '1018': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1018.png',
 '1026': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1026.png',
 '1027': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1027.png',
 '1028': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1028.png',
 '1029': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1029.png',
 '1031': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1031.png',
 '1033': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1033.png',
 '1035': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1035.png',
 '1036': 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1036.png',

# ----------------------------------------------------------------------------------------------------------

# 2. 각 챔피언마다 시작아이템의 픽률과 승률

In [5]:
champ_games = df.groupby("CHAMPIONNAME")[["GAMEID"]].count().rename(columns = {"GAMEID":"GAMES"}).reset_index()

In [8]:
starting_item_games = df.groupby(["CHAMPIONNAME","STARTING_ITEM"])[["WIN"]].count().rename(columns={"WIN":"STARTING_ITEMS_GAMES"}).reset_index()

In [9]:
starting_item_games

,CHAMPIONNAME,STARTING_ITEM,STARTING_ITEMS_GAMES
0,Aatrox,1001|1006|2051,22
1,Aatrox,1001|1006|3177,3
2,Aatrox,1001|1028|3057,1
3,Aatrox,1001|1029|3067,1
4,Aatrox,1001|1036|1036|1036|2003,1
...,...,...,...
15296,Zyra,3070|3108,11
15297,Zyra,3070|3112,22
15298,Zyra,3070|3916,2
15299,Zyra,3070|4642,3


In [22]:
starting_item_win = df.groupby(["CHAMPIONNAME","STARTING_ITEM"])[["WIN"]].sum().reset_index()

In [23]:
starting_item_win

,CHAMPIONNAME,STARTING_ITEM,WIN
0,Aatrox,1001|1006|2051,8
1,Aatrox,1001|1006|3177,2
2,Aatrox,1001|1028|3057,0
3,Aatrox,1001|1029|3067,0
4,Aatrox,1001|1036|1036|1036|2003,1
...,...,...,...
15296,Zyra,3070|3108,5
15297,Zyra,3070|3112,8
15298,Zyra,3070|3916,1
15299,Zyra,3070|4642,3


In [26]:
starting_item_win_pick = pd.merge(pd.merge(starting_item_games, starting_item_win, on=["CHAMPIONNAME","STARTING_ITEM"]),champ_games, on="CHAMPIONNAME")

In [27]:
starting_item_win_pick

,CHAMPIONNAME,STARTING_ITEM,STARTING_ITEMS_GAMES,WIN,GAMES
0,Aatrox,1001|1006|2051,22,8,2878
1,Aatrox,1001|1006|3177,3,2,2878
2,Aatrox,1001|1028|3057,1,0,2878
3,Aatrox,1001|1029|3067,1,0,2878
4,Aatrox,1001|1036|1036|1036|2003,1,1,2878
...,...,...,...,...,...
15296,Zyra,3070|3108,11,5,1764
15297,Zyra,3070|3112,22,8,1764
15298,Zyra,3070|3916,2,1,1764
15299,Zyra,3070|4642,3,3,1764


In [31]:
starting_item_win_pick["PICK_RATE"] = round(starting_item_win_pick.STARTING_ITEMS_GAMES / starting_item_win_pick.GAMES * 100,1)

In [32]:
starting_item_win_pick["WIN_RATE"] = round(starting_item_win_pick.WIN / starting_item_win_pick.STARTING_ITEMS_GAMES * 100,1)

In [36]:
starting_item_win_pick = starting_item_win_pick.sort_values("PICK_RATE", ascending=False)

In [37]:
starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == "Aatrox"]

,CHAMPIONNAME,STARTING_ITEM,STARTING_ITEMS_GAMES,WIN,GAMES,PICK_RATE,WIN_RATE
22,Aatrox,1001|6029,606,279,2878,21.1,46.0
12,Aatrox,1001|2031|2051,323,187,2878,11.2,57.9
17,Aatrox,1001|3044,218,110,2878,7.6,50.5
14,Aatrox,1001|2031|3177,194,105,2878,6.7,54.1
101,Aatrox,2031|6029,168,85,2878,5.8,50.6
...,...,...,...,...,...,...,...
46,Aatrox,1028|3158,1,0,2878,0.0,0.0
40,Aatrox,1028|2003|3158,1,0,2878,0.0,0.0
37,Aatrox,1028|2003|2003|2003|3067,1,1,2878,0.0,100.0
51,Aatrox,1029|2003|2051,1,0,2878,0.0,0.0


In [54]:
tmp = []
for i in champ_lst:
    tmp_lst = []
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["STARTING_ITEM"])
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["PICK_RATE"])
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["WIN_RATE"])
    
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[1]["STARTING_ITEM"])
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[1]["PICK_RATE"])
    tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[1]["WIN_RATE"])
    
    tmp.append(tmp_lst)

In [55]:
champ_starting_item = pd.DataFrame(tmp,columns=["CHAMPIONNAME","F_STARTING_ITEM","F_PICK_RATE","F_WIN_RATE","S_STARTING_ITEM","S_PICK_RATE","S_WIN_RATE"])

In [56]:
champ_starting_item

,CHAMPIONNAME,F_STARTING_ITEM,F_PICK_RATE,F_WIN_RATE,S_STARTING_ITEM,S_PICK_RATE,S_WIN_RATE
0,Aatrox,1001|6029,21.1,46.0,1001|2031|2051,11.2,57.9
1,Ahri,2003|2003|3802,62.1,42.9,3802,18.5,48.4
2,Akali,2003|2003|4635,28.4,42.5,1001|2003|3145,18.3,39.5
3,Akshan,2003|2003|6670,36.1,42.4,1001|2031|3184,19.2,47.0
4,Alistar,2003|2051|3070,20.7,55.6,1001|2031|2051,14.1,53.9
...,...,...,...,...,...,...,...
152,Zed,1001|3134,37.8,45.3,2031|3134,17.4,45.3
153,Ziggs,2003|2003|3802,51.9,54.5,3802,15.8,55.8
154,Zilean,2003|2003|3802,37.9,42.1,2003|3070|3112,9.7,43.0
155,Zoe,2003|2003|3802,63.0,44.7,3802,19.3,44.5


In [57]:
def starting_item_img(row):
    tmp = []
    tmp += list(map(lambda x: item_lst[x], row["F_STARTING_ITEM"].split("|")))
    tmp += [str(row.F_PICK_RATE)]
    tmp += [str(row.F_WIN_RATE)]
    
    tmp += list(map(lambda x: item_lst[x], row["S_STARTING_ITEM"].split("|")))
    tmp += [str(row.S_PICK_RATE)]
    tmp += [str(row.S_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [74]:
champ_starting_item["STARTING_ITEM_IMG"] = champ_starting_item.apply(lambda x: starting_item_img(x), axis=1)

In [83]:
champ_starting_item.iloc[105]["STARTING_ITEM_IMG"].split("|")

['http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1001.png',
 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1036.png',
 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/1036.png',
 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/3070.png',
 '20.2',
 '57.1',
 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/2003.png',
 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/2003.png',
 'http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/6670.png',
 '14.5',
 '52.1']

# 3. 위 작업의 함수화

In [26]:
def starting_item_img_one(row):
    tmp = []
    tmp += list(map(lambda x: item_lst[x], row["F_STARTING_ITEM"].split("|")))
    tmp += [str(row.F_PICK_RATE)]
    tmp += [str(row.F_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [27]:
def starting_item_img_two(row):
    tmp = []
    tmp += list(map(lambda x: item_lst[x], row["S_STARTING_ITEM"].split("|")))
    tmp += [str(row.S_PICK_RATE)]
    tmp += [str(row.S_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [86]:
url = "http://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/item.json"
res = requests.get(url).json()
item_key = list(res['data'].keys())
itemurl = list(map(lambda x: "http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/"+x+".png", item_key))
item_lst = dict(zip(item_key,itemurl))

In [28]:
def starting_item(df):
    champ_games = df.groupby("CHAMPIONNAME")[["GAMEID"]].count().rename(columns = {"GAMEID":"GAMES"}).reset_index()
    starting_item_games = df.groupby(["CHAMPIONNAME","STARTING_ITEM"])[["WIN"]].count().rename(columns={"WIN":"STARTING_ITEMS_GAMES"}).reset_index()
    starting_item_win = df.groupby(["CHAMPIONNAME","STARTING_ITEM"])[["WIN"]].sum().reset_index()
    starting_item_win_pick = pd.merge(pd.merge(starting_item_games, starting_item_win, on=["CHAMPIONNAME","STARTING_ITEM"]),champ_games, on="CHAMPIONNAME")
    starting_item_win_pick["PICK_RATE"] = round(starting_item_win_pick.STARTING_ITEMS_GAMES / starting_item_win_pick.GAMES * 100,1)
    starting_item_win_pick["WIN_RATE"] = round(starting_item_win_pick.WIN / starting_item_win_pick.STARTING_ITEMS_GAMES * 100,1)
    starting_item_win_pick = starting_item_win_pick.sort_values("PICK_RATE", ascending=False)
    
    tmp = []
    for i in champ_lst:
        tmp_lst = []
        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["STARTING_ITEM"])
        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["PICK_RATE"])
        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[0]["WIN_RATE"])

        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[1]["STARTING_ITEM"])
        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[1]["PICK_RATE"])
        tmp_lst.append(starting_item_win_pick[starting_item_win_pick.CHAMPIONNAME == i].iloc[1]["WIN_RATE"])

        tmp.append(tmp_lst)
        
    champ_starting_item = pd.DataFrame(tmp,columns=["CHAMPIONNAME","F_STARTING_ITEM","F_PICK_RATE","F_WIN_RATE","S_STARTING_ITEM","S_PICK_RATE","S_WIN_RATE"])

    champ_starting_item["STARTING_ITEM_IMG_ONE"] = champ_starting_item.apply(lambda x: starting_item_img_one(x), axis=1)
    
    champ_starting_item["STARTING_ITEM_IMG_TWO"] = champ_starting_item.apply(lambda x: starting_item_img_two(x), axis=1)

    return champ_starting_item

In [29]:
champ_starting_item = starting_item(df)

In [31]:
champ_starting_item

,CHAMPIONNAME,F_STARTING_ITEM,F_PICK_RATE,F_WIN_RATE,S_STARTING_ITEM,S_PICK_RATE,S_WIN_RATE,STARTING_ITEM_IMG_ONE,STARTING_ITEM_IMG_TWO
0,Aatrox,1001|6029,21.5,46.7,1001|2031|2051,11.4,57.7,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
1,Ahri,2003|2003|3802,62.3,43.9,3802,18.8,47.4,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
2,Akali,2003|2003|4635,28.2,42.9,1001|2003|3145,18.0,39.8,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
3,Akshan,2003|2003|6670,36.2,42.0,1001|2031|3184,19.2,47.1,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
4,Alistar,2003|2051|3070,20.5,56.5,1001|2031|2051,14.3,54.7,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
...,...,...,...,...,...,...,...,...,...
152,Zed,1001|3134,38.1,44.9,2031|3134,17.5,44.3,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
153,Ziggs,2003|2003|3802,52.6,54.6,3802,15.4,56.9,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
154,Zilean,2003|2003|3802,37.7,41.3,2003|3070|3112,9.6,43.6,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
155,Zoe,2003|2003|3802,63.3,44.2,3802,19.2,44.8,http://ddragon.leagueoflegends.com/cdn/11.24.1...,http://ddragon.leagueoflegends.com/cdn/11.24.1...
